In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



# Import API key
from api_keys import g_key

merge CSV files into a combined dataframe for analysis

In [20]:
file = "output/my_combined.csv"

county_df = pd.read_csv(file)
county_df.head()

file2 = "Resources/countycoordinates.xlsx"

coord_df = pd.read_excel(file2)
clean_coord = coord_df[["State", "County", "Latitude", "Longitude"]]
clean_coord

data_loc = pd.DataFrame.merge(county_df, clean_coord, on=["State", "County"])



#data_loc.to_csv('output/clean_with_lat_long.csv', index = False)

data_loc.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,# Deaths,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,830.0,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,2573.0,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,413.0,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,407.0,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,906.0,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371


In [22]:
#add additional data column to dataframe and merge again with medicare data

file = "output/clean_with_lat_long.csv"

longlat_df = pd.read_csv(file)
longlat_df.head()

file2 = "Resources/Medicare_Enrollment_2017.csv"

health_df = pd.read_csv(file2)

clean_data2 = pd.DataFrame.merge(longlat_df, health_df, on=["State", "County"])



#clean_data2.to_csv('output/clean_data2.csv', index = False)

clean_data2.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749,5784
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840,28388
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197,4372
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439,2480
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371,6070


In [23]:
#print column to see data

clean_data2["Food Environment Index"]


0       6.9
1       7.5
2       4.9
3       7.3
4       8.3
       ... 
3088    8.1
3089    8.0
3090    7.5
3091    8.0
3092    8.0
Name: Food Environment Index, Length: 3093, dtype: float64

In [24]:


clean_data2 

#create inverse values so that the heat layer map shows green for access to food and red for no access.  

clean_data2["Inverse Index"] = clean_data2["Food Environment Index"].apply(lambda x: abs(x - 10))

clean_data2["Inverse Index"]

0       3.1
1       2.5
2       5.1
3       2.7
4       1.7
       ... 
3088    1.9
3089    2.0
3090    2.5
3091    2.0
3092    2.0
Name: Inverse Index, Length: 3093, dtype: float64

In [25]:
#print column headers to easy viewing when analyzing data
for col in clean_data2.columns:
    print(col)

State
County
POP2010
OHU2010
PovertyRate
MedianFamilyIncome
LILATracts_1And10
LILATracts_halfAnd10
LILATracts_1And20
LILATracts_Vehicle
LAPOP1_10
LAPOP05_10
LAPOP1_20
LALOWI1_10
LALOWI05_10
LALOWI1_20
lapophalf
TractKids
TractSeniors
TractWhite
TractBlack
TractAsian
TractNHOPI
TractAIAN
TractOMultir
TractHispanic
TractHUNV
TractSNAP
Food Environment Index
# Deaths
Years of Potential Life Lost Rate
% Obese
% Fair/Poor
# Diabetics
# Violent Crimes
# Unemployed
% Children in Poverty
Latitude
Longitude
Original Medicare
Inverse Index


Create heat map of US with data by county showing the Food Environment Index Inverse 0 is best, 10 is worst 

In [26]:

locations = clean_data2[["Latitude", "Longitude"]]

food_env = clean_data2["Inverse Index"]

In [27]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=food_env, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

create a heat map to show obesity % for US by county. 100% is red, 0% is faint green

In [28]:
locations = data_loc[["Latitude", "Longitude"]]

obese = data_loc["% Obese"].astype(float)

In [29]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=obese, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

create heat maps by State Montana. create dataframe of Just Montana

In [30]:
montana = clean_data2.loc[clean_data2["State"] == "Montana",:]

montana.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare,Inverse Index
1585,Montana,Beaverhead,9246,4014,14.566667,53953.0,2,2,2,1,...,24,12,126.0,10.0,171.0,21.0,45.132700,-112.898820,1841,2.7
1586,Montana,Big Horn,12865,4004,31.220000,46813.2,4,4,1,1,...,39,22,193.0,28.0,372.0,40.0,45.423382,-107.489853,1607,5.5
1587,Montana,Blaine,6491,2357,29.800000,38925.5,1,1,1,1,...,36,20,100.0,10.0,120.0,40.0,48.432676,-108.958602,1004,4.7
1588,Montana,Broadwater,5612,2347,11.250000,48507.5,0,0,0,0,...,30,12,64.0,20.0,124.0,16.0,46.332424,-111.495487,1111,2.7
1589,Montana,Carbon,10078,4571,11.480000,60412.6,1,1,0,0,...,24,11,131.0,20.0,200.0,15.0,45.227386,-109.028016,1993,2.5


In [32]:
#heat map of inverse data of Food Environment for Montana

locations2 = montana[["Latitude", "Longitude"]]


food_env2 = montana["Inverse Index"]

In [33]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=food_env2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
#create heat map for Obese % for Montana

locations = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
obese2 = montana["% Obese"]

In [53]:

# Plot Heatmap
fig = gmaps.figure()
marker_locations = montana[["Latitude", "Longitude"]]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=obese2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer

fig.add_layer(heat_layer)
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:

# Using the template add the Obese % marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Obesity %: </dt><dd>{% Obese}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
obese_info = [info_box_template1.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

In [69]:
# Plot Heatmap
fig = gmaps.figure()
marker_locations = montana[["Latitude", "Longitude"]]

# Create heat layer
heat_layer2 = gmaps.heatmap_layer(locations2, weights=obese2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.25)


# Add layer
marker_layer2 = gmaps.marker_layer(locations, info_box_content=obese_info)

fig.add_layer(heat_layer2)
fig.add_layer(markers)
fig.add_layer(marker_layer2)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:

gmaps.configure(api_key= g_key)

marker_locations = montana[["Latitude", "Longitude"]]
    

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
#Display figure
fig



Figure(layout=FigureLayout(height='420px'))

In [43]:

# Using the template add the food environment marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Food Environment Index: </dt><dd>{Food Environment Index}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
county_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

In [44]:

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=county_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

heat_layer = gmaps.heatmap_layer(locations2, weights=food_env2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)

fig.add_layer(heat_layer)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))

In [50]:

gmaps.configure(api_key= g_key)

marker_locations = montana[["Latitude", "Longitude"]]
    

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:

# Using the template add the food environment Index marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt>Food Environment Index: </dt><dd>{Food Environment Index}</dd>
</dl>
"""


# NOTE: be sure to update with your DataFrame name
county_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

# Diabetics

In [55]:
locations = data_loc[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
diabetic = data_loc["# Diabetics"].astype(float)

In [56]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=diabetic, 
                                 dissipating=False, max_intensity=3000,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [58]:
locations = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
diabetic2 = montana["# Diabetics"]

In [65]:

# Using the template add the diabetic # marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt># of Diabetics: </dt><dd>{# Diabetics}</dd>
</dl>
"""


# NOTE: be sure to update with your DataFrame name
diabetic_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]


In [67]:
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=diabetic2, 
                                 dissipating=False, max_intensity=800,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
marker_layer = gmaps.marker_layer(locations, info_box_content=diabetic_info)
fig.add_layer(marker_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Unemployed

In [73]:
locations = data_loc[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
unemployed = data_loc["# Unemployed"].astype(float)

In [74]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=unemployed, 
                                 dissipating=False, max_intensity=3000,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [75]:
locations = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
unemployed2 = montana["# Unemployed"]

In [76]:
# Using the template add the # unemployed marks to the heatmap
info_box_template = """
<dl>
<dt>County: </dt><dd>{County}</dd>
<dt># of Unemployed: </dt><dd>{# Unemployed}</dd>
</dl>
"""

# NOTE: be sure to update with your DataFrame name
employmemt_info = [info_box_template.format(**row) for index, row in montana.iterrows()]
locations = montana[["Latitude", "Longitude"]]

In [78]:
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=unemployed2, 
                                 dissipating=False, max_intensity=800,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
marker_layer = gmaps.marker_layer(locations, info_box_content=employmemt_info)
fig.add_layer(marker_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))